In [1]:
pip install open3d

# 1. Point Cloud Registration Using ICP

## Import Dataset

In [2]:
import zipfile
import os

# Path to the zip file
zip_path = "kitti_sample.zip"
# Destination path for extraction
extract_path = "kitti_sample"

# Check if the zip file exists
if os.path.exists(zip_path):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path)
    print(f"Extracted to {extract_path}")
else:
    print(f"Zip file not found at: {zip_path}")


Extracted to kitti_sample


## Show Lidar Points

In [3]:
import numpy as np
import open3d as o3d

# Path to the point cloud binary file
bin_path = "kitti_sample/kitti_sample/00/000001.bin"

# Read data from the binary file
point_cloud = np.fromfile(bin_path, dtype=np.float32).reshape(-1, 4)
points = point_cloud[:, :3]  # Only XYZ coordinates

# Print the total number of points
print(f"Total number of points in the file: {points.shape[0]}")

# Create a PointCloud object using Open3D
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(points)

# Visualize the point cloud using Open3D
o3d.visualization.draw_geometries([pcd], window_name="LIDAR Frame 000001")


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
Total number of points in the file: 124605


## Show Lidar Points Intensity

In [4]:
import matplotlib.pyplot as plt

# Read data from the binary file
point_cloud = np.fromfile(bin_path, dtype=np.float32).reshape(-1, 4)
points = point_cloud[:, :3]        # XYZ coordinates
intensities = point_cloud[:, 3]    # Intensity values

# Print the total number of points
print(f"Total number of points in the file: {points.shape[0]}")

# Normalize intensity values between 0 and 1
intensity_normalized = (intensities - intensities.min()) / (intensities.ptp() + 1e-8)

# Apply a colormap (e.g., viridis) to the normalized intensities
colormap = plt.get_cmap("plasma")
colors = colormap(intensity_normalized)[:, :3]  # Use only RGB (ignore alpha)

# Create a PointCloud object using Open3D
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(points)
pcd.colors = o3d.utility.Vector3dVector(colors)

# Visualize the point cloud using Open3D
o3d.visualization.draw_geometries([pcd], window_name="LIDAR Frame 000001 (Colored by Intensity)")

Total number of points in the file: 124605


## (a) Select Two Point Clouds:

In [5]:
def load_bin_to_pcd(bin_path):
    """Reads .bin file and converts to Open3D PointCloud"""
    points = np.fromfile(bin_path, dtype=np.float32).reshape(-1, 4)[:, :3]
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points)
    return pcd

# File paths (relative to project directory)
base_path = "kitti_sample/kitti_sample/00/"
source_pcd = load_bin_to_pcd(base_path + "000001.bin")
target_pcd = load_bin_to_pcd(base_path + "000005.bin")

# Downsampling for speed and noise reduction
voxel_size = 0.5  # Try 0.2 if needed
source_down = source_pcd.voxel_down_sample(voxel_size)
target_down = target_pcd.voxel_down_sample(voxel_size)

# Estimate normals (needed for point-to-plane ICP)
source_down.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=1.0, max_nn=30))
target_down.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=1.0, max_nn=30))


## (b) Visualize Unregistered Point Clouds:

In [6]:
def plot_before_icp_o3d(source, target, title="Before ICP - Initial Alignment"):
    # Create copies of the point clouds to avoid modifying the original data
    source_temp = o3d.geometry.PointCloud(source)
    target_temp = o3d.geometry.PointCloud(target)

    # Assign colors to each point cloud for visual distinction
    source_temp.paint_uniform_color([1, 0, 0])  # Red
    target_temp.paint_uniform_color([0, 0, 1])  # Blue

    # Visualize with a title (note: the title is not displayed inside the window)
    o3d.visualization.draw_geometries([source_temp, target_temp],
                                      window_name=title)

# Run the visualization function
plot_before_icp_o3d(source_down, target_down)


## (c) Apply ICP Algorithm:

In [7]:
# ICP Point-to-Point
icp_result = o3d.pipelines.registration.registration_icp(
    source_down,  # Source point cloud
    target_down,  # Target point cloud
    max_correspondence_distance=1.0,  # Maximum correspondence threshold
    init=np.eye(4),  # Initial transformation (identity)
    estimation_method=o3d.pipelines.registration.TransformationEstimationPointToPoint()
)

# Results
print("Transformation matrix (ICP):\n", icp_result.transformation)
print("Fitness:", icp_result.fitness)
print("Inlier RMSE:", icp_result.inlier_rmse)

Transformation matrix (ICP):
 [[ 0.99990311  0.01380751  0.00176822 -1.03774161]
 [-0.01380418  0.99990295 -0.00188165  0.00165326]
 [-0.00179403  0.00185706  0.99999667  0.00996909]
 [ 0.          0.          0.          1.        ]]
Fitness: 0.7087866108786611
Inlier RMSE: 0.5061625714170965


In [8]:
# Extract translation
translation = icp_result.transformation[:3, 3]
tx, ty, tz = translation

# Extract rotation matrix
R = icp_result.transformation[:3, :3]

# Convert rotation matrix to Euler angles (roll, pitch, yaw)
# Assuming intrinsic Tait-Bryan angles in ZYX order (yaw-pitch-roll)
import math

sy = math.sqrt(R[0,0] ** 2 + R[1,0] ** 2)
singular = sy < 1e-6

if not singular:
    roll = math.atan2(R[2,1], R[2,2])
    pitch = math.atan2(-R[2,0], sy)
    yaw = math.atan2(R[1,0], R[0,0])
else:
    roll = math.atan2(-R[1,2], R[1,1])
    pitch = math.atan2(-R[2,0], sy)
    yaw = 0

# Convert to degrees
roll_deg = math.degrees(roll)
pitch_deg = math.degrees(pitch)
yaw_deg = math.degrees(yaw)

# Print results
print(f"Translation: tx={tx:.3f} m, ty={ty:.3f} m, tz={tz:.3f} m")
print(f"Rotation (degrees): roll={roll_deg:.2f}°, pitch={pitch_deg:.2f}°, yaw={yaw_deg:.2f}°")


Translation: tx=-1.038 m, ty=0.002 m, tz=0.010 m
Rotation (degrees): roll=0.11°, pitch=0.10°, yaw=-0.79°


## (d) Visualize Aligned Point Clouds:

In [9]:
def plot_after_icp_o3d(source, target, transformation, title="After ICP - Aligned Point Clouds"):
    # Apply transformation to a copy of the source point cloud
    source_transformed = o3d.geometry.PointCloud(source)
    source_transformed.transform(transformation)

    # Assign colors to each point cloud
    source_transformed.paint_uniform_color([1, 0, 0])  # Red (Source)
    target.paint_uniform_color([0, 0, 1])              # Blue (Target)

    # Visualize in Open3D
    o3d.visualization.draw_geometries([source_transformed, target],
                                      window_name=title)

# Run the visualization function
plot_after_icp_o3d(source_down, target_down, icp_result.transformation)


In [10]:
import copy

def plot_before_after_icp_one_window(source, target, transformation):
    # Make deep copies to keep original point clouds unchanged
    source_before = copy.deepcopy(source)
    source_after = copy.deepcopy(source)
    target_copy = copy.deepcopy(target)

    # Apply the transformation to the post-ICP version
    source_after.transform(transformation)

    # Color the point clouds
    source_before.paint_uniform_color([1, 0, 0])   # Red - Before ICP
    source_after.paint_uniform_color([0, 1, 0])    # Green - After ICP
    target_copy.paint_uniform_color([0, 0, 1])     # Blue - Target

    # Visualize all in one window
    o3d.visualization.draw_geometries([source_before, source_after, target_copy],
                                      window_name="Before & After ICP - One Window")

# Run the function
plot_before_after_icp_one_window(source_down, target_down, icp_result.transformation)


## Animation

In [11]:
import time

def animate_icp_alignment(source, target, transformation, steps=100, delay=0.02):
    # Copy and color clouds
    source_copy = copy.deepcopy(source)
    target_copy = copy.deepcopy(target)

    source_copy.paint_uniform_color([1, 0, 0])   # Red - Source
    target_copy.paint_uniform_color([0, 0, 1])   # Blue - Target

    # Setup visualizer
    vis = o3d.visualization.Visualizer()
    vis.create_window(window_name="ICP Animation", width=960, height=720)
    vis.add_geometry(source_copy)
    vis.add_geometry(target_copy)

    # Decompose the transformation
    delta = np.linalg.inv(np.eye(4)) @ transformation  # from identity to final
    t_final = transformation[:3, 3]
    R_final = transformation[:3, :3]

    for i in range(steps + 1):
        alpha = i / steps

        # Interpolate translation linearly
        t_interp = alpha * t_final

        # Interpolate rotation linearly (not physically accurate but works visually)
        R_interp = np.eye(3) + alpha * (R_final - np.eye(3))

        # Build interpolated transformation matrix
        T_interp = np.eye(4)
        T_interp[:3, :3] = R_interp
        T_interp[:3, 3] = t_interp

        # Transform and update source
        source_step = copy.deepcopy(source)
        source_step.transform(T_interp)
        source_copy.points = source_step.points  # update points in-place

        vis.update_geometry(source_copy)
        vis.poll_events()
        vis.update_renderer()
        time.sleep(delay)

    vis.run()
    vis.destroy_window()


In [12]:
animate_icp_alignment(source_down, target_down, icp_result.transformation)


# 2. Incremental Registration of Multiple Point Clouds Using ICP

## (a) Use the Result from Previous Part:
## (b) Visualize the Final Map:

In [1]:
import os
import numpy as np
import open3d as o3d
import copy
import random

# Paths
base_path = "kitti_sample/00/"

# Parameters
voxel_size = 0.2
icp_threshold = 1.5
max_iter = 2000

def load_bin_to_pcd(bin_path):
    points = np.fromfile(bin_path, dtype=np.float32).reshape(-1, 4)[:, :3]
    pcd = o3d.geometry.PointCloud(o3d.utility.Vector3dVector(points))
    return pcd

def do_icp(source, target, threshold, init, max_iter):
    source.estimate_normals(o3d.geometry.KDTreeSearchParamHybrid(radius=0.5, max_nn=30))
    target.estimate_normals(o3d.geometry.KDTreeSearchParamHybrid(radius=0.5, max_nn=30))
    return o3d.pipelines.registration.registration_icp(
        source, target, threshold, init,
        o3d.pipelines.registration.TransformationEstimationPointToPlane(),
        o3d.pipelines.registration.ICPConvergenceCriteria(max_iteration=max_iter)
    )

# Load all .bin files
bin_files = sorted([f for f in os.listdir(base_path) if f.endswith(".bin")])
pc_list = [load_bin_to_pcd(os.path.join(base_path, f)) for f in bin_files]

# Build unified map
unified_map = copy.deepcopy(pc_list[0])
cumulative_transform = np.eye(4)

for i in range(1, len(pc_list)):
    source = pc_list[i - 1].voxel_down_sample(voxel_size)
    target = pc_list[i].voxel_down_sample(voxel_size)

    print(f"Registering frame {i} to frame {i-1}...")
    reg = do_icp(target, source, icp_threshold, np.eye(4), max_iter)
    cumulative_transform = cumulative_transform @ reg.transformation

    transformed = copy.deepcopy(pc_list[i])
    transformed.transform(cumulative_transform)
    transformed.paint_uniform_color([random.random(), random.random(), random.random()])
    unified_map += transformed

# Downsample and visualize
final_map = unified_map.voxel_down_sample(voxel_size * 2)
o3d.visualization.draw_geometries([final_map], window_name="Unified Map")


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
Registering frame 1 to frame 0...
Registering frame 2 to frame 1...
Registering frame 3 to frame 2...
Registering frame 4 to frame 3...
Registering frame 5 to frame 4...
Registering frame 6 to frame 5...
Registering frame 7 to frame 6...
Registering frame 8 to frame 7...
Registering frame 9 to frame 8...
Registering frame 10 to frame 9...
Registering frame 11 to frame 10...
Registering frame 12 to frame 11...
Registering frame 13 to frame 12...
Registering frame 14 to frame 13...
Registering frame 15 to frame 14...
Registering frame 16 to frame 15...
Registering frame 17 to frame 16...
Registering frame 18 to frame 17...
Registering frame 19 to frame 18...
Registering frame 20 to frame 19...
Registering frame 21 to frame 20...
Registering frame 22 to frame 21...
Registering frame 23 to frame 22...
Registering frame 24 t

## Map and Trajectory

In [15]:
import os
import numpy as np
import open3d as o3d
import copy
import random

# Paths
base_path = "kitti_sample/00/"
gt_pose_path = "kitti_sample/gt_pose_00.txt"

# Parameters
voxel_size = 0.2
icp_threshold = 1.5
max_iter = 2000

# --- Helper functions ---

def load_bin_to_pcd(bin_path):
    pts = np.fromfile(bin_path, dtype=np.float32).reshape(-1, 4)[:, :3]
    return o3d.geometry.PointCloud(o3d.utility.Vector3dVector(pts))

def do_icp(source, target, threshold, init, max_iter):
    source.estimate_normals(o3d.geometry.KDTreeSearchParamHybrid(radius=0.5, max_nn=30))
    target.estimate_normals(o3d.geometry.KDTreeSearchParamHybrid(radius=0.5, max_nn=30))
    return o3d.pipelines.registration.registration_icp(
        source, target, threshold, init,
        o3d.pipelines.registration.TransformationEstimationPointToPlane(),
        o3d.pipelines.registration.ICPConvergenceCriteria(max_iteration=max_iter)
    )

def transform_gt_pose(pose):
    
    R_rect = np.array([[0, 0, 1],
                       [-1, 0, 0],
                       [0, -1, 0]])
    T = np.eye(4)
    T[:3, :3] = R_rect @ pose[:3, :3]
    T[:3, 3] = R_rect @ pose[:3, 3]
    return T

# --- Load point clouds ---
bin_files = sorted([f for f in os.listdir(base_path) if f.endswith(".bin")])
pc_list = [load_bin_to_pcd(os.path.join(base_path, f)) for f in bin_files]

# --- Load ground truth poses ---
gt_poses = []
with open(gt_pose_path, 'r') as f:
    for line in f:
        pose = np.fromstring(line, sep=' ').reshape(3, 4)
        mat = np.eye(4)
        mat[:3, :4] = pose
        mat = transform_gt_pose(mat)
        gt_poses.append(mat)

# --- Mapping ---
unified_map = copy.deepcopy(pc_list[0])
unified_map.paint_uniform_color([0.6, 0.6, 0.6])
cumulative_transform = np.eye(4)
trajectory_points = [gt_poses[0][:3, 3]]

for i in range(1, len(pc_list)):
    source = pc_list[i - 1].voxel_down_sample(voxel_size)
    target = pc_list[i].voxel_down_sample(voxel_size)
    
    reg = do_icp(target, source, icp_threshold, np.eye(4), max_iter)
    cumulative_transform = cumulative_transform @ reg.transformation

    transformed = copy.deepcopy(pc_list[i])
    transformed.transform(cumulative_transform)
    transformed.paint_uniform_color([random.random(), random.random(), random.random()])
    unified_map += transformed

    if i < len(gt_poses):
        trajectory_points.append(gt_poses[i][:3, 3])

# --- Final downsample ---
final_map = unified_map.voxel_down_sample(voxel_size * 2)

# --- Visualize map + GT trajectory ---
if len(trajectory_points) >= 2:
    lines = [[i, i + 1] for i in range(len(trajectory_points) - 1)]
    gt_line = o3d.geometry.LineSet()
    gt_line.points = o3d.utility.Vector3dVector(trajectory_points)
    gt_line.lines = o3d.utility.Vector2iVector(lines)
    gt_line.colors = o3d.utility.Vector3dVector([[0, 1, 0]] * len(lines))
    o3d.visualization.draw_geometries([final_map, gt_line],
                                      window_name="Map + Ground Truth Trajectory",
                                      width=1200, height=900)
else:
    o3d.visualization.draw_geometries([final_map], window_name="Map Only")
